In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Drama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Представление и предобработка текстовых данных 

1.1 Операции по предобработке:
* токенизация
* стемминг / лемматизация
* удаление стоп-слов
* удаление пунктуации
* приведение к нижнему регистру
* любые другие операции над текстом

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [4]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

Реализовать функцию `preprocess_text(text: str)`, которая:
* приводит строку к нижнему регистру
* заменяет все символы, кроме a-z, A-Z и знаков .,!? на пробел

In [7]:
def preprocess_text(text):
    text = text.lower()
    
    text = re.sub(r'[^a-zA-Z.,!?]', ' ', text)
    
    return text

In [9]:
preprocess_text(text)

'select your preferences and run the install command. stable represents the most currently tested and supported version of pytorch. note that libtorch is only available for c  '

1.2 Представление текстовых данных при помощи бинарного кодирования


Представить первое предложение из `text` в виде тензора `sentence_t`: `sentence_t[i] == 1`, если __слово__ с индексом `i` присуствует в предложении.

In [11]:
def binary_arr(words, _word_bag):
    return [(word in _word_bag) * 1 for word in words]


word_bag = list(set(preprocess_text(text).split()))
sentence_1 = binary_arr(text.split('.')[0], word_bag)

print(word_bag)
print(sentence_1)

['and', 'install', 'represents', 'most', 'version', 'run', 'tested', 'available', 'select', 'your', 'only', 'command.', 'pytorch.', 'c', 'preferences', 'currently', 'note', 'that', 'of', 'stable', 'for', 'supported', 'is', 'the', 'libtorch']
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


## 2. Классификация фамилий по национальности

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)

2.5 Реализовать класс `SurnamesDataset`

2.6. Обучить классификатор.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [13]:
# Задание 1

surnames = pd.read_csv('surnames.csv')
surnames.head(5)

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


In [15]:
# Задание 2

nationalities = surnames.nationality.unique()
nationality_to_idx = dict(zip(nationalities, range(len(nationalities))))
idx_to_nationalities = {v: u for u, v in nationality_to_idx.items()}
idx_to_nationalities

{0: 'English',
 1: 'French',
 2: 'Arabic',
 3: 'Russian',
 4: 'Japanese',
 5: 'Chinese',
 6: 'Italian',
 7: 'Czech',
 8: 'Irish',
 9: 'German',
 10: 'Greek',
 11: 'Spanish',
 12: 'Polish',
 13: 'Dutch',
 14: 'Vietnamese',
 15: 'Korean',
 16: 'Portuguese',
 17: 'Scottish'}

In [16]:
# Задание 3
surnames_train, surnames_test = train_test_split(surnames, test_size=0.2)

In [17]:
# Задание 4
class Vocab:
    def __init__(self, data: pd.DataFrame):
        data = data[['surname']].drop_duplicates()  # оставляем уникальные фамилии

        letters = data.surname.str.lower().str.split('').apply(lambda x: x[1:-1])  # разбиваем фамилии по буквам
        letters = set(letters.sum())  # находим все буквы

        self.idx_to_token = dict(zip(range(1, len(letters) + 1), letters))  # id - буква
        self.idx_to_token[0] = ''

        self.token_to_idx = dict(zip(letters, range(1, len(letters) + 1)))  # буква - id
        self.token_to_idx[''] = 0

        self.vocab_len = len(self.idx_to_token)


vocab = Vocab(surnames)
vocab.vocab_len, vocab.token_to_idx['z'], vocab.idx_to_token[1]

(56, 28, 'o')

In [18]:
MAX_SURNAME_LEN = surnames.surname.str.len().max()
MAX_SURNAME_LEN

17

In [19]:
# Задание 5
class SurnamesDataset(Dataset):
    def __init__(self, x, y, _vocab: Vocab):
        self.x = x
        self.y = y
        self.vocab = _vocab

    def vectorize(self, _surname: str):
        """ Генерирует представление фамилии surname в при помощи бинарного кодирования (см. 1.2) """
        output_vector = [0 for _ in range(MAX_SURNAME_LEN)]
        _surname = _surname.lower()
        for l in range(len(_surname)):
            output_vector[MAX_SURNAME_LEN - len(_surname) + l] = self.vocab.token_to_idx[_surname[l]]

        return torch.FloatTensor(output_vector)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.vectorize(self.x.iloc[idx]).long(), torch.tensor(nationality_to_idx[self.y.iloc[idx]])

## 3. Классификация обзоров ресторанов

Датасет: https://disk.yandex.ru/d/nY1o70JtAuYa8g

3.1 Считать файл `yelp/raw_train.csv`. Оставить от исходного датасета 10% строчек.

3.2 Воспользоваться функцией `preprocess_text` из 1.1 для обработки текста отзыва. Закодировать рейтинг числами, начиная с 0.

3.3 Разбить датасет на обучающую и тестовую выборку

3.4 Реализовать класс `Vocab` (токен = слово)

3.5 Реализовать класс `ReviewDataset`

3.6 Обучить классификатор

3.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)


In [ ]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = ...
    self.token_to_idx = ...
    self.vocab_len = ...

In [ ]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
    '''Генерирует представление отзыва review при помощи бинарного кодирования (см. 1.2)'''
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return ...